In [1]:
import sys

sys.path.insert(0, '../Lab/')
sys.path.insert(0, '../New_Hybrids/')
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from UserIcmKNNCFRecommender import UserIcmKNNCFRecommender
from ItemIcmKNNCFRecommender import ItemIcmKNNCFRecommender
from WeightedHybridV2 import WeightedHybridScoreRecommender
from optimize_weights import optimize_weights
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.IALSRecommender import IALSRecommender
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
import matplotlib
import matplotlib.pyplot as plt
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python

%matplotlib inline 

import numpy as np
sys.path.insert(0, '../datasets/')
from load_tables import load_ICM_csr, load_URM_csr
def plotta(t,maps):
    fig, ax = plt.subplots()
    ax.plot(t, maps)

    ax.set(xlabel='T', ylabel='MAP')
    ax.grid()
    plt.show()

In [2]:
ICM_all = load_ICM_csr()
URM_all = load_URM_csr()

# PARAMS ALE

In [4]:
userknn = {}
userknn["topK"] = 102
userknn["shrink"] = 1

itemknn = {}
itemknn["topK"] = 141
itemknn["shrink"] = 47


uicm = {}
uicm["topK"] = 181
uicm["shrink"] = 0.1
uicm["normalize"] = True

iicm = {}
iicm["topK"] = 893
iicm["shrink"] = 2
iicm["normalize"] = True

alpha = {}
alpha["alpha"] = 0.547615508822564
alpha["topK"] = 500

beta= {}
beta["topK"]=500
beta["alpha"]=0.3784740936494376
beta["beta"]=0.1
beta["implicit"]=False
beta["normalize_similarity"]=False

ials={}
ials["epochs"]=45
ials["num_factors"]=50
ials["alpha"]=10.0
ials["epsilon"]=10.0
ials["reg"]=1e-07
ials["init_mean"]=1.0
ials["init_std"]=0.6773548197826565

only_URM = {
    "URM_train" : None
}

also_ICM = {
    "URM_train" : None,
    "ICM" : ICM_all
}


In [5]:
ale_recs =  [
    P3alphaRecommender,
    ItemKNNCFRecommender,
    UserKNNCFRecommender,
    UserIcmKNNCFRecommender,
    RP3betaRecommender,
    ItemIcmKNNCFRecommender,
    IALSRecommender 
    ]
ale_fits = [alpha, itemknn, userknn,uicm,beta,iicm,ials]
ale_inits = [only_URM,only_URM,only_URM,also_ICM,only_URM,also_ICM,only_URM]
ale_weights = [
    0.7890828279804322,
    0.0001,
    0.31527729065553595,
    0.12162462193350662,
    0.06519442814723547,
    1.0,
    0.3445924625940507
]

# PARAMS ENRICO

In [7]:
cy = {}
cy["epochs"] = 400
cy["topK"] = 900
cy["positive_threshold_BPR"] = 7e-18
cy["learning_rate"] = 0.0002
cy["batch_size"] = 10
cy["sgd_mode"] = "sdg"

cython = {
    "URM_train" : None,
    "recompile_cython" : False,
    "verbose" : False
}


alpha = {}
alpha["alpha"] = 0.3784740936494376
alpha["topK"] = 500

In [8]:
enrico_recs = [
UserKNNCFRecommender,
P3alphaRecommender,
ItemIcmKNNCFRecommender,
UserIcmKNNCFRecommender,
ItemKNNCFRecommender,
RP3betaRecommender,
SLIM_BPR_Cython
]

enrico_fits = [userknn,alpha,iicm,uicm,itemknn,beta,cy]
enrico_inits = [only_URM,only_URM,also_ICM,also_ICM,only_URM,only_URM,cython]
enrico_weights = [1, 0.79488, 2.76, 0.144, 1.5, 0.294, 0.05]

# IBRIDO DI IBRIDI

In [10]:
recs_ibrid = [WeightedHybridScoreRecommender,WeightedHybridScoreRecommender]
ibrid_ale_init = {
    "URM_train" : None,
    "recs" : ale_recs,
    "inits" : ale_inits
}
ibrid_enrico_init = {
    "URM_train" : None,
    "recs" : enrico_recs,
    "inits" : enrico_inits
}

ibrid_inits = [ibrid_ale_init, ibrid_enrico_init]

ibrid_ale_fit = {}
ibrid_ale_fit["weights"] = ale_weights
ibrid_ale_fit["fits"] = ale_fits


ibrid_enrico_fit = {}
ibrid_enrico_fit["weights"] = enrico_weights
ibrid_enrico_fit["fits"] = enrico_fits


ibrid_fits = [ibrid_ale_fit,ibrid_enrico_fit]


In [ ]:
w = optimize_weights(URM_all,7,recs_ibrid,ibrid_inits,ibrid_fits,levels=4,weights = None, index = 0)


optimize 1
WeightedHybridScoreRecommender: URM Detected 42 (0.53 %) cold users.
WeightedHybridScoreRecommender: URM Detected 2030 (7.82 %) cold items.
WeightedHybridScoreRecommender: URM Detected 42 (0.53 %) cold users.
WeightedHybridScoreRecommender: URM Detected 2030 (7.82 %) cold items.
P3alphaRecommender: URM Detected 42 (0.53 %) cold users.
P3alphaRecommender: URM Detected 2030 (7.82 %) cold items.
ItemKNNCFRecommender: URM Detected 42 (0.53 %) cold users.
ItemKNNCFRecommender: URM Detected 2030 (7.82 %) cold items.
UserKNNCFRecommender: URM Detected 42 (0.53 %) cold users.
UserKNNCFRecommender: URM Detected 2030 (7.82 %) cold items.
UserKNNCFRecommender: URM Detected 42 (0.53 %) cold users.
UserKNNCFRecommender: URM Detected 2030 (7.82 %) cold items.
UserKNNCFRecommender: URM Detected 42 (0.53 %) cold users.
UserKNNCFRecommender: URM Detected 29 (0.14 %) cold items.
RP3betaRecommender: URM Detected 42 (0.53 %) cold users.
RP3betaRecommender: URM Detected 2030 (7.82 %) cold items.

SLIM_BPR_Recommender: Epoch 18 of 400. Elapsed time 3.59 sec
SLIM_BPR_Recommender: Epoch 19 of 400. Elapsed time 3.66 sec
SLIM_BPR_Recommender: Epoch 20 of 400. Elapsed time 3.74 sec
SLIM_BPR_Recommender: Epoch 21 of 400. Elapsed time 3.83 sec
SLIM_BPR_Recommender: Epoch 22 of 400. Elapsed time 3.90 sec
SLIM_BPR_Recommender: Epoch 23 of 400. Elapsed time 3.97 sec
SLIM_BPR_Recommender: Epoch 24 of 400. Elapsed time 4.03 sec
SLIM_BPR_Recommender: Epoch 25 of 400. Elapsed time 4.10 sec
SLIM_BPR_Recommender: Epoch 26 of 400. Elapsed time 4.17 sec
SLIM_BPR_Recommender: Epoch 27 of 400. Elapsed time 4.27 sec
SLIM_BPR_Recommender: Epoch 28 of 400. Elapsed time 4.44 sec
SLIM_BPR_Recommender: Epoch 29 of 400. Elapsed time 4.55 sec
SLIM_BPR_Recommender: Epoch 30 of 400. Elapsed time 4.66 sec
SLIM_BPR_Recommender: Epoch 31 of 400. Elapsed time 4.82 sec
SLIM_BPR_Recommender: Epoch 32 of 400. Elapsed time 5.07 sec
SLIM_BPR_Recommender: Epoch 33 of 400. Elapsed time 5.15 sec
SLIM_BPR_Recommender: Ep

SLIM_BPR_Recommender: Epoch 152 of 400. Elapsed time 10.68 sec
SLIM_BPR_Recommender: Epoch 153 of 400. Elapsed time 10.72 sec
SLIM_BPR_Recommender: Epoch 154 of 400. Elapsed time 10.76 sec
SLIM_BPR_Recommender: Epoch 155 of 400. Elapsed time 10.80 sec
SLIM_BPR_Recommender: Epoch 156 of 400. Elapsed time 10.84 sec
SLIM_BPR_Recommender: Epoch 157 of 400. Elapsed time 10.88 sec
SLIM_BPR_Recommender: Epoch 158 of 400. Elapsed time 10.92 sec
SLIM_BPR_Recommender: Epoch 159 of 400. Elapsed time 10.96 sec
SLIM_BPR_Recommender: Epoch 160 of 400. Elapsed time 11.00 sec
SLIM_BPR_Recommender: Epoch 161 of 400. Elapsed time 11.04 sec
SLIM_BPR_Recommender: Epoch 162 of 400. Elapsed time 11.08 sec
SLIM_BPR_Recommender: Epoch 163 of 400. Elapsed time 11.12 sec
SLIM_BPR_Recommender: Epoch 164 of 400. Elapsed time 11.16 sec
SLIM_BPR_Recommender: Epoch 165 of 400. Elapsed time 11.20 sec
SLIM_BPR_Recommender: Epoch 166 of 400. Elapsed time 11.24 sec
SLIM_BPR_Recommender: Epoch 167 of 400. Elapsed time 11

SLIM_BPR_Recommender: Epoch 283 of 400. Elapsed time 16.32 sec
SLIM_BPR_Recommender: Epoch 284 of 400. Elapsed time 16.39 sec
SLIM_BPR_Recommender: Epoch 285 of 400. Elapsed time 16.45 sec
SLIM_BPR_Recommender: Epoch 286 of 400. Elapsed time 16.50 sec
SLIM_BPR_Recommender: Epoch 287 of 400. Elapsed time 16.57 sec
SLIM_BPR_Recommender: Epoch 288 of 400. Elapsed time 16.63 sec
SLIM_BPR_Recommender: Epoch 289 of 400. Elapsed time 16.67 sec
SLIM_BPR_Recommender: Epoch 290 of 400. Elapsed time 16.71 sec
SLIM_BPR_Recommender: Epoch 291 of 400. Elapsed time 16.75 sec
SLIM_BPR_Recommender: Epoch 292 of 400. Elapsed time 16.79 sec
SLIM_BPR_Recommender: Epoch 293 of 400. Elapsed time 16.84 sec
SLIM_BPR_Recommender: Epoch 294 of 400. Elapsed time 16.87 sec
SLIM_BPR_Recommender: Epoch 295 of 400. Elapsed time 16.91 sec
SLIM_BPR_Recommender: Epoch 296 of 400. Elapsed time 16.95 sec
SLIM_BPR_Recommender: Epoch 297 of 400. Elapsed time 17.00 sec
SLIM_BPR_Recommender: Epoch 298 of 400. Elapsed time 17

--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 7735.79 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 8262.48 column/sec, elapsed time 0.02 min
Similarity column 7947 ( 100 % ), 1283.34 column/sec, elapsed time 0.10 min
--------FITTING IN PROGRESS: RP3betaRecommender-------
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 2923.93 column/sec, elapsed time 0.15 min
Similarity column 25975 ( 100 % ), 1644.45 column/sec, elapsed time 0.26 min
--------FITTING IN PROGRESS: IALSRecommender-------
IALSRecommender: Epoch 1 of 45. Elapsed time 8.56 sec
IALSRecommender: Epoch 2 of 45. Elapsed time 17.79 sec
IALSRecommender: Epoch 3 of 45. Elapsed time 26.59 sec
IALSRecommender: Epoch 4 of 45. Elapsed time 35.57 sec
IALSRecommender: Epoch 5 of 45. Elapsed time 45.06 sec
IALSRecommender: Epoch 6 of 45. Elapsed time 54.81 sec
IALSRecomme